In [1]:
import ast
import json
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import pickle

import warnings
warnings.filterwarnings('ignore')

from collections import Counter
from random import randint, random
from scipy.sparse import coo_matrix, hstack
from sklearn.metrics.pairwise import euclidean_distances, cosine_distances, cosine_similarity


In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
interactions_df = pd.read_csv('../data_original/interactions.csv')
users_df = pd.read_csv('../data_original/users.csv')
items_df = pd.read_csv('../data_original/items.csv')

In [4]:
interactions_df['t_dat'] = pd.to_datetime(interactions_df['last_watch_dt'], format="%Y-%m-%d")
interactions_df['timestamp'] = interactions_df.t_dat.values.astype(np.int64) // 10 ** 9

In [5]:
df = interactions_df[['user_id', 'item_id', 'timestamp']].rename(
    columns={'user_id': 'user_id:token', 'item_id': 'item_id:token', 'timestamp': 'timestamp:float'})

In [6]:
!mkdir recbox_data

In [7]:
df.to_csv('recbox_data/recbox_data.inter', index=False, sep='\t')

In [12]:
import logging
from logging import getLogger
from recbole.config import Config
from recbole.data import create_dataset, data_preparation
from recbole.model.sequential_recommender import GRU4Rec, Caser
from recbole.trainer import Trainer
from recbole.utils import init_seed, init_logger
from recbole.quick_start import run_recbole

In [13]:
parameter_dict = {
    'data_path': '',
    'USER_ID_FIELD': 'user_id',
    'ITEM_ID_FIELD': 'item_id',
    'TIME_FIELD': 'timestamp',
    'device': 'GPU',
    'user_inter_num_interval': "[40,inf)",
    'item_inter_num_interval': "[40,inf)",
    'load_col': {'inter': ['user_id', 'item_id', 'timestamp']},
    'neg_sampling': None,
    'epochs': 10,
    'eval_args': {
        'split': {'RS': [9, 0, 1]},
        'group_by': 'user',
        'order': 'TO',
        'mode': 'full'}
}
config = Config(model='MultiVAE', dataset='recbox_data', config_dict=parameter_dict)

# init random seed
init_seed(config['seed'], config['reproducibility'])

# logger initialization
init_logger(config)
logger = getLogger()
# Create handlers
c_handler = logging.StreamHandler()
c_handler.setLevel(logging.INFO)
logger.addHandler(c_handler)

# write config info into log
# logger.info(config)

In [14]:
dataset = create_dataset(config)
logger.info(dataset)

24 Dec 18:55    INFO  recbox_data
The number of users: 13355
Average actions of users: 63.815710648494836
The number of items: 3294
Average actions of items: 258.78985727300335
The number of inters: 852195
The sparsity of the dataset: 98.06281322904924%
Remain Fields: ['user_id', 'item_id', 'timestamp']
recbox_data
The number of users: 13355
Average actions of users: 63.815710648494836
The number of items: 3294
Average actions of items: 258.78985727300335
The number of inters: 852195
The sparsity of the dataset: 98.06281322904924%
Remain Fields: ['user_id', 'item_id', 'timestamp']


In [15]:
# dataset splitting
train_data, valid_data, test_data = data_preparation(config, dataset)

24 Dec 18:55    INFO  [Training]: train_batch_size = [2048] train_neg_sample_args: [{'distribution': 'uniform', 'sample_num': 1, 'alpha': 1.0, 'dynamic': False, 'candidate_num': 0}]
[Training]: train_batch_size = [2048] train_neg_sample_args: [{'distribution': 'uniform', 'sample_num': 1, 'alpha': 1.0, 'dynamic': False, 'candidate_num': 0}]
24 Dec 18:55    INFO  [Evaluation]: eval_batch_size = [4096] eval_args: [{'split': {'RS': [9, 0, 1]}, 'order': 'TO', 'group_by': 'user', 'mode': {'valid': 'full', 'test': 'full'}}]
[Evaluation]: eval_batch_size = [4096] eval_args: [{'split': {'RS': [9, 0, 1]}, 'order': 'TO', 'group_by': 'user', 'mode': {'valid': 'full', 'test': 'full'}}]


In [16]:
import time

In [20]:
%%time
model_list = ["BPR", "CDAE", "MultiVAE"] 

for model_name in model_list:
    print(f"running {model_name}...")
    start = time.time()
    result = run_recbole(model=model_name, dataset = 'recbox_data', config_dict = parameter_dict)
    t = time.time() - start
    print(f"It took {t/60:.2f} mins")
    print(result)

running BPR...


24 Dec 19:06    INFO  ['/home/projects/RecoService/.venv/lib/python3.10/site-packages/ipykernel_launcher.py', '--f=/home/dbashkirov/.local/share/jupyter/runtime/kernel-v2-1514935szZNWB4pdQ0t.json']
['/home/projects/RecoService/.venv/lib/python3.10/site-packages/ipykernel_launcher.py', '--f=/home/dbashkirov/.local/share/jupyter/runtime/kernel-v2-1514935szZNWB4pdQ0t.json']
24 Dec 19:06    INFO  
General Hyper Parameters:
gpu_id = 0
use_gpu = True
seed = 2020
state = INFO
reproducibility = True
data_path = recbox_data
checkpoint_dir = saved
show_progress = True
save_dataset = False
dataset_save_path = None
save_dataloaders = False
dataloaders_save_path = None
log_wandb = False

Training Hyper Parameters:
epochs = 10
train_batch_size = 2048
learner = adam
learning_rate = 0.001
train_neg_sample_args = {'distribution': 'uniform', 'sample_num': 1, 'alpha': 1.0, 'dynamic': False, 'candidate_num': 0}
eval_step = 1
stopping_step = 10
clip_grad_norm = None
weight_decay = 0.0
loss_decimal_place = 

It took 2.11 mins
{'best_valid_score': -inf, 'valid_score_bigger': True, 'best_valid_result': None, 'test_result': OrderedDict([('recall@10', 0.0824), ('mrr@10', 0.1716), ('ndcg@10', 0.0819), ('hit@10', 0.3479), ('precision@10', 0.0457)])}
running CDAE...


24 Dec 19:08    INFO  ['/home/projects/RecoService/.venv/lib/python3.10/site-packages/ipykernel_launcher.py', '--f=/home/dbashkirov/.local/share/jupyter/runtime/kernel-v2-1514935szZNWB4pdQ0t.json']
['/home/projects/RecoService/.venv/lib/python3.10/site-packages/ipykernel_launcher.py', '--f=/home/dbashkirov/.local/share/jupyter/runtime/kernel-v2-1514935szZNWB4pdQ0t.json']
24 Dec 19:08    INFO  
General Hyper Parameters:
gpu_id = 0
use_gpu = True
seed = 2020
state = INFO
reproducibility = True
data_path = recbox_data
checkpoint_dir = saved
show_progress = True
save_dataset = False
dataset_save_path = None
save_dataloaders = False
dataloaders_save_path = None
log_wandb = False

Training Hyper Parameters:
epochs = 10
train_batch_size = 2048
learner = adam
learning_rate = 0.001
train_neg_sample_args = {'distribution': 'uniform', 'sample_num': 1, 'alpha': 1.0, 'dynamic': False, 'candidate_num': 0}
eval_step = 1
stopping_step = 10
clip_grad_norm = None
weight_decay = 0.0
loss_decimal_place = 

It took 1.30 mins
{'best_valid_score': -inf, 'valid_score_bigger': True, 'best_valid_result': None, 'test_result': OrderedDict([('recall@10', 0.0634), ('mrr@10', 0.1439), ('ndcg@10', 0.0659), ('hit@10', 0.2723), ('precision@10', 0.0344)])}
running MultiVAE...


24 Dec 19:09    INFO  ['/home/projects/RecoService/.venv/lib/python3.10/site-packages/ipykernel_launcher.py', '--f=/home/dbashkirov/.local/share/jupyter/runtime/kernel-v2-1514935szZNWB4pdQ0t.json']
['/home/projects/RecoService/.venv/lib/python3.10/site-packages/ipykernel_launcher.py', '--f=/home/dbashkirov/.local/share/jupyter/runtime/kernel-v2-1514935szZNWB4pdQ0t.json']
24 Dec 19:09    INFO  
General Hyper Parameters:
gpu_id = 0
use_gpu = True
seed = 2020
state = INFO
reproducibility = True
data_path = recbox_data
checkpoint_dir = saved
show_progress = True
save_dataset = False
dataset_save_path = None
save_dataloaders = False
dataloaders_save_path = None
log_wandb = False

Training Hyper Parameters:
epochs = 10
train_batch_size = 2048
learner = adam
learning_rate = 0.001
train_neg_sample_args = {'distribution': 'uniform', 'sample_num': 1, 'alpha': 1.0, 'dynamic': False, 'candidate_num': 0}
eval_step = 1
stopping_step = 10
clip_grad_norm = None
weight_decay = 0.0
loss_decimal_place = 

It took 1.52 mins
{'best_valid_score': -inf, 'valid_score_bigger': True, 'best_valid_result': None, 'test_result': OrderedDict([('recall@10', 0.084), ('mrr@10', 0.1695), ('ndcg@10', 0.0825), ('hit@10', 0.3503), ('precision@10', 0.0467)])}
CPU times: user 9min 15s, sys: 25.5 s, total: 9min 41s
Wall time: 4min 55s


{'best_valid_score': -inf, 'valid_score_bigger': True, 'best_valid_result': None, 'test_result': OrderedDict([('recall@10', 0.0824), ('mrr@10', 0.1716), ('ndcg@10', 0.0819), ('hit@10', 0.3479), ('precision@10', 0.0457)])}

{'best_valid_score': -inf, 'valid_score_bigger': True, 'best_valid_result': None, 'test_result': OrderedDict([('recall@10', 0.0634), ('mrr@10', 0.1439), ('ndcg@10', 0.0659), ('hit@10', 0.2723), ('precision@10', 0.0344)])}

{'best_valid_score': -inf, 'valid_score_bigger': True, 'best_valid_result': None, 'test_result': OrderedDict([('recall@10', 0.084), ('mrr@10', 0.1695), ('ndcg@10', 0.0825), ('hit@10', 0.3503), ('precision@10', 0.0467)])}

Самый лучший результат у MultiVAE